In [31]:
using PCT

## $x^{\dagger} A x$

In [32]:
f, _ = @pct (A::Her) -> 𝒫((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j))); f

(A) -> 
    𝒫((x) -> 
        ∑((i, j), (x(i)'⋅x(j)⋅A(i, j))))

In [33]:
df = redux(vdiff(f), settings=symmetry_settings)

(A) -> 
    (x) -> 
        (_d) -> 
            (∑((j), (A(j, _d)'⋅x(j)))⋅2.0)

## Trace of Product

In [34]:
f, _ = @pct begin
    tr = (X::CM) -> sum(i, X(i, i))
    matmul = (A::CM, B::CM) -> (i::N, j::N) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let 
    tr = (X) -> 
        ∑((i), X(i, i))
    matmul = (A, B) -> 
        (i, j) -> 
            ∑((n), (A(i, n)⋅B(n, j)))
    𝒫((A, B) -> 
        tr((matmul(A, B)+matmul(B, A))))
end

In [35]:
redux(vdiff(eval_all(f)); settings=symmetry_settings)

(A, B) -> 
    (_d, _d_1) -> 
        (B(_d_1, _d)'⋅2.0), (_d, _d_1) -> 
        (A(_d_1, _d)'⋅2.0)

## Conjugate Gradient

In [36]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::N) -> x(i) + α * (r(i) + β * p(i))))
end; f

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i), (x(i)⋅b(i)))+∑((i, j), (x(i)⋅x(j)⋅A(i, j)⋅0.5)))
        𝒫((α, β) -> 
            E((i) -> 
                ((((p(i)⋅β)+r(i))⋅α)+x(i))))
    end

In [10]:
df = vdiff(f)

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i), (x(i)⋅b(i)))+∑((i, j), (x(i)⋅x(j)⋅A(i, j)⋅0.5)))
        (α, β) -> 
            ∑((i), (((p(i)⋅β)+r(i))⋅𝒫(E)((i) -> 
                ((((p(i)⋅β)+r(i))⋅α)+x(i)), 1)(i))), (∑((i), (𝒫(E)((i) -> 
                ((((p(i)⋅β)+r(i))⋅α)+x(i)), 1)(i)⋅p(i)))⋅α)
    end

In [11]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings)

(A, r, p, b, x) -> 
    (α, β) -> 
        (∑((i), (((p(i)⋅β)+r(i))⋅b(i)))+∑((i, j), (((((p(j)⋅β)+r(j))⋅α)+x(j))⋅((p(i)⋅β)+r(i))⋅A(j, i)))), ((∑((i), (b(i)⋅p(i)))+∑((i, j), (((((p(j)⋅β)+r(j))⋅α)+x(j))⋅p(i)⋅A(i, j))))⋅α)

## Lagrangian Mechanics

In [12]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [13]:
f, _ = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [14]:
redux(vdiff(f); settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        (D'((_d) -> 
            𝒫((_z_1) -> 
                L(r(_d), _z_1))(D(r)(_d), 1))+(_d) -> 
            𝒫((_z_2) -> 
                L(_z_2, D(r)(_d)))(r(_d), 1))

## Hartree Fock Gradient and Hessian Vector Product

In [40]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [41]:
f, _ = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s))); f

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), (C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i))))

In [42]:
df = redux(vdiff(f); settings=symmetry_settings)

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            (∑((j, p, r, s), (J(r, s, p, _d)'⋅C(s, j)'⋅C(p, _d_1)⋅C(r, j)))⋅4.0)

In [43]:
df |> get_body |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_a_2, _a_3) -> 
        ((∑((_d, _d_1, p, s), (C(p, _d_1)'⋅J(_a_2, s, p, _d)⋅_k(_d, _d_1)⋅C(s, _a_3)))+∑((_d, _d_1, p, r), (J(r, _a_2, p, _d)'⋅_k(_d, _d_1)'⋅C(p, _d_1)⋅C(r, _a_3)))+∑((_d, j, r, s), (C(r, j)'⋅J(r, s, _a_2, _d)⋅_k(_d, _a_3)⋅C(s, j))))⋅4.0)

## Foster Boys

In [45]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> begin 
        r = (i::N) -> sum((p, q), R(p, q) * C(i, p)' * C(i, q))
        sum(i, sum((p, q), R2(p, q) * C(i, p)' * C(i, q)) - r(i)' * r(i))
    end); f

(R, R2) -> 
    𝒫((C) -> 
        let 
            r = (i) -> 
                ∑((p, q), (R(p, q)⋅C(i, q)⋅C(i, p)'))
            ∑((i), (∑((p, q), (R2(p, q)⋅C(i, q)⋅C(i, p)'))+(r(i)⋅r(i)'⋅-1.0)))
        end)

In [46]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings)

(R, R2) -> 
    (C) -> 
        (_d, _d_1) -> 
            (2.0⋅(∑((p), (R2(_d_1, p)⋅C(_d, p)))+∑((_i, p, q), (R(_d_1, p)⋅R(_i, q)⋅C(_d, q)⋅C(_d, p)⋅C(_d, _i)'⋅-2.0))))

## Localized Wannier Functions

In [37]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (N, N, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> C
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (N, N, BZ) -> C
    end
end;

In [38]:
f, _ = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::N, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), ρ(n, b)' * ρ(n, b))
    end); f

(S, w) -> 
    𝒫((U) -> 
        let 
            ρ = (n, b) -> 
                ∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))
            ∑((n, b), (ρ(n, b)'⋅ρ(n, b)))
        end)

In [39]:
# df = vdiff(eval_all(f); settings=custom_settings(:logging=>true; preset = symmetry_settings))
df = redux(vdiff(eval_all(f)); settings=symmetry_settings)

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            (∑((b, p, k, _p, q), (U(_p, _d_1, k)'⋅S(q, _p, (b+k), k)'⋅S(p, _d, (_d_2+(b⋅-1.0)), _d_2)'⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (_d_2+(b⋅-1.0)))))⋅4.0)

In [22]:
_, ctx = @pct begin
    @space Tau begin
        type=(R, RV) -> RM
    end
    
    @space Nu begin
        type=(RV, ) -> R
    end
        
    @space S begin
        type=(RV, )->RV
    end
end;

In [14]:
f, _ = @pct _ ctx (τ::Tau, ν::Nu, p::S) -> :vdiff(pullback((x::RV) -> τ(ν(x), p(x)))); f

(τ, ν, p) -> 
    __vdiff(𝒫((x) -> 
        τ(ν(x), p(x))))

In [15]:
df = process_directive(f)

(τ, ν, p) -> 
    (x) -> 
        (𝒫(ν)(x, 𝒫((_z) -> 
            τ(_z, p(x)))(ν(x), 1))+𝒫(p)(x, 𝒫((_z_1) -> 
            τ(ν(x), _z_1))(p(x), 1)))